In [1]:
#Import packages
import pandas as pd
import numpy as np
import os

In [2]:
#Read in BF locations data
BF_Locations = pd.read_csv(os.path.join("https://download.data.world/s/a5gdzsmxenwcwqmnbjaaizbfk4uea3"))

In [3]:
#Read in BF reports data
BF_Reports = pd.read_csv(os.path.join("https://download.data.world/s/td52m5guv2cpruwjo6pr2cd3fbk5cx"))

In [4]:
#Read in UFO data
UFO = pd.read_csv(os.path.join("https://download.data.world/s/lobqhnvklhbppb4ayke4b23lsupg57"))

In [5]:
#Change column name
BF_Locations = BF_Locations.rename(columns={'number': 'report_number'})

In [6]:
#Merge two BF datasets
BF = pd.merge(BF_Locations, BF_Reports, how='left', left_on='report_number', right_on='report_number')

In [7]:
#Split 'title' column by report number and report
BF[['report_number2', 'report']] = BF.pop('title').str.split(pat=':', n=1, expand=True)

In [8]:
#Drop redundant 'report_number2' column
BF = BF.drop(columns= ['report_number2'])

In [9]:
#Reformatting timestamp column
BF['timestamp'] = pd.to_datetime(BF['timestamp'], format = "%Y-%m-%dT%H:%M:%S.%f")

In [10]:
#Splitting timestamp column
BF['report_date'] = BF['timestamp'].dt.date
BF['time'] = BF['timestamp'].dt.time

In [11]:
#Removing unnecessary columns
BF = BF.drop(['timestamp', 'year', 'season', 'report_class', 'month', 'date', 'a_g_references', 'time'], axis=1)

In [12]:
#Drop unnecessary columns
UFO = UFO.drop(['stats', 'report_link'], axis = 1)

In [13]:
#Remove all countries other than USA
UFO = UFO.loc[UFO["country"] == "USA"]

In [14]:
#Split 'date_time' column
UFO[['date', 'time']] = UFO.pop('date_time').str.split(pat=' ', n=1, expand=True)

In [15]:
BF.to_excel(r'C:\Users\geens\Desktop\Data\BF.xlsx', index=False, header=True)

In [17]:
writer = pd.ExcelWriter(r'C:\Users\geens\Desktop\Data\UFO.xlsx')
UFO.to_excel(writer)